# **Koneksi Ke Google Big Query**

In [ ]:
from google.colab import auth
from google.colab import data_table
from google.cloud import bigquery
from pandas_gbq import to_gbq
import pandas as pd
import numpy as np

# PRoses autentikasi akun
auth.authenticate_user()
print('Authenticated')

Authenticated


In [5]:
from google.cloud import bigquery

# Buat BigQuery client
project_id = 'dq-komdigi-projectdata'
bq_client = bigquery.Client(project = project_id)

## **Proses Ekstraksi Data dari sumber (Extract)**

In [56]:
%%bigquery data_transaksi --project {project_id}

-- Tulis Query disini
SELECT * FROM dqlab-468906.dqcommerce.tbl_transaction

Query is running:   0%|          |

Downloading:   0%|          |

Setelah query dijalankan maka akan disimpan sebagai dataframe pandas

In [84]:
%%bigquery data_product --project {project_id}

-- Tulis Query disini
SELECT * FROM dqlab-468906.dqcommerce.tbl_product

Query is running:   0%|          |

Downloading:   0%|          |

In [85]:
%%bigquery data_funnel --project {project_id}

-- Tulis Query disini
SELECT * FROM dqlab-468906.dqcommerce.tbl_funnel

Query is running:   0%|          |

Downloading:   0%|          |

## **Periksa Informasi Umum pada Data**

In [36]:
#Periksa info pada data
data_transaksi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829262 entries, 0 to 829261
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   trx_id      829257 non-null  object
 1   product_id  829262 non-null  object
 2   trx_date    829262 non-null  object
 3   trx_time    829262 non-null  dbtime
 4   units       829251 non-null  Int64 
dtypes: Int64(1), dbtime(1), object(3)
memory usage: 32.4+ MB


In [14]:
data_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_id        37 non-null     object
 1   product_name      37 non-null     object
 2   product_category  37 non-null     object
 3   product_cost      37 non-null     object
 4   product_price     37 non-null     object
dtypes: object(5)
memory usage: 1.6+ KB


In [15]:
data_funnel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15950 entries, 0 to 15949
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         15950 non-null  Int64 
 1   product_id   15950 non-null  object
 2   purchase     15950 non-null  Int64 
 3   add_to_cart  15950 non-null  Int64 
 4   click        15950 non-null  Int64 
 5   view         15950 non-null  Int64 
dtypes: Int64(5), object(1)
memory usage: 825.7+ KB


## **Proses Transformasi Data (Transform)**

In [26]:
data_transaksi.sample(10)

,date,product_id,purchase,add_to_cart,click,view
6458,21012025,DQProduk-014,46,493,1502,2859
12018,9082025,DQProduk-026,11,120,434,543
15879,14082024,DQProduk-035,14,119,387,635
8449,11012025,DQProduk-018,29,268,1505,1920
10607,24032025,DQProduk-023,44,464,2105,3069
6349,9042024,DQProduk-014,34,354,1909,2680
6814,31082024,DQProduk-015,7,65,340,657
7976,12022025,DQProduk-017,18,167,698,838
3798,14092025,DQProduk-008,94,897,4641,8372
10256,2092025,DQProduk-022,14,126,554,737


In [30]:
#Ubah kolom trx_date dari integer menjadi string
data_transaksi['trx_date'] = data_transaksi['trx_date'].astype('str')
#Buat Kondisi ketika data panjangnya 7 maka biarkan, jika tidak 7 tambahkan 0 didepannya
data_transaksi['trx_date'] = np.where(data_transaksi['trx_date'].str.len() == 7, '0' + data_transaksi['trx_date'], data_transaksi['trx_date'])

data_transaksi

,trx_id,product_id,trx_date,trx_time,units
0,DQTrx829214,DQProduk-016,30092025,01:18:05,<NA>
1,DQTrx829166,DQProduk-017,30092025,18:58:33,<NA>
2,DQTrx829118,DQProduk-018,30092025,06:58:09,<NA>
3,DQTrx829186,DQProduk-019,30092025,09:28:13,<NA>
4,DQTrx829212,DQProduk-019,30092025,15:00:55,<NA>
...,...,...,...,...,...
829257,DQTrx401952,DQProduk-030,20122024,08:40:50,24
829258,DQTrx184262,DQProduk-022,18062024,00:01:18,25
829259,DQTrx041748,DQProduk-014,12022024,22:03:16,27
829260,DQTrx042347,DQProduk-014,12022024,15:38:55,30


In [51]:
data_transaksi.sample(10)

,trx_id,product_id,trx_date,trx_time,units
781514,DQTrx150095,DQProduk-018,21052024,07:16:52,3
729347,DQTrx032167,DQProduk-025,03022024,01:09:49,2
326279,DQTrx795711,DQProduk-013,07092025,16:51:09,1
441508,DQTrx416733,DQProduk-019,29122024,07:42:39,1
577211,DQTrx145914,DQProduk-027,18052024,19:20:31,1
779283,DQTrx673892,DQProduk-015,16062025,12:57:44,3
533444,DQTrx235406,DQProduk-025,03082024,08:36:13,1
327397,DQTrx020351,DQProduk-014,22012024,01:51:04,1
218480,DQTrx306656,DQProduk-008,09102024,21:34:36,1
751439,DQTrx439858,DQProduk-003,14012025,22:30:31,3


In [49]:
#Ubah kolom trx_date dari integer menjadi string
data_transaksi['trx_date'] = data_transaksi['trx_date'].astype('str')
#Buat Kondisi ketika data panjangnya 7 maka biarkan, jika tidak 7 tambahkan 0 didepannya
data_transaksi['trx_date'] = np.where(data_transaksi['trx_date'].str.len() == 7, '0' + data_transaksi['trx_date'], data_transaksi['trx_date'])

data_transaksi.sample(10)

,trx_id,product_id,trx_date,trx_time,units
815517,DQTrx712271,DQProduk-010,09072025,07:40:16,4
453896,DQTrx625173,DQProduk-019,16052025,03:19:09,1
245836,DQTrx725053,DQProduk-008,17072025,04:40:02,1
118613,DQTrx036037,DQProduk-006,06022024,07:02:33,1
121108,DQTrx054080,DQProduk-006,25022024,00:12:44,1
582451,DQTrx306794,DQProduk-027,09102024,11:24:20,1
194630,DQTrx013703,DQProduk-008,15012024,06:07:52,1
411135,DQTrx293736,DQProduk-018,28092024,04:25:56,1
526076,DQTrx105227,DQProduk-025,14042024,10:11:09,1
627328,DQTrx661963,DQProduk-030,08062025,23:40:02,1


In [ ]:
#merubah format tanggal menjadi standar iso

In [53]:
data_transaksi['trx_date'] = pd.to_datetime(data_transaksi['trx_date'], format='%d%m%Y')
data_transaksi.sample(10)

,trx_id,product_id,trx_date,trx_time,units
411180,DQTrx294592,DQProduk-018,2024-09-29,04:48:00,1
536284,DQTrx276223,DQProduk-025,2024-09-12,23:42:24,1
488557,DQTrx825917,DQProduk-021,2025-09-29,10:23:57,1
518084,DQTrx739722,DQProduk-024,2025-07-27,10:26:10,1
277475,DQTrx112805,DQProduk-010,2024-04-21,09:50:34,1
548655,DQTrx511811,DQProduk-025,2025-03-06,01:48:19,1
699535,DQTrx001929,DQProduk-006,2024-01-02,20:10:21,2
127056,DQTrx110215,DQProduk-006,2024-04-18,06:45:29,1
544300,DQTrx424376,DQProduk-025,2025-01-03,10:42:21,1
554238,DQTrx618927,DQProduk-025,2025-05-12,14:27:20,1


In [54]:
#cek missing value
data_transaksi['trx_id'].isna().sum()

np.int64(5)

In [60]:
#cek missing value dengan memperlihatkan data/ melihat nilai kosong didalam kolom
data_transaksi[data_transaksi['trx_id'].isna()]
#cara handling ada 3 lidrop, imputasi
#metode mana yang harus diterapkan tergantung konteks data, jk data seperti ini trx id kosong artinya sistem gagal mengenerate, hanya imputan fraud atau kegagalan sistem, user menyatakan data seperti ini tidak valid ssehingg harus drop data

,trx_id,product_id,trx_date,trx_time,units
106825,None,DQProduk-004,1012024,14:43:09,1
193154,None,DQProduk-008,1012024,09:27:56,1
468999,None,DQProduk-021,1012024,13:16:09,1
499044,None,DQProduk-024,1012024,06:32:49,1
571924,None,DQProduk-027,1012024,04:15:36,1


In [64]:
#data asli sebelum di drop
data_transaksi.shape


(829257, 5)

In [66]:
#hapus data yang kosong
data_transaksi = data_transaksi[data_transaksi['trx_id'].notnull()]
data_transaksi.shape

(829257, 5)

In [69]:
data_transaksi[data_transaksi['units'].isna()]

,trx_id,product_id,trx_date,trx_time,units
0,DQTrx829214,DQProduk-016,30092025,01:18:05,<NA>
1,DQTrx829166,DQProduk-017,30092025,18:58:33,<NA>
2,DQTrx829118,DQProduk-018,30092025,06:58:09,<NA>
3,DQTrx829186,DQProduk-019,30092025,09:28:13,<NA>
4,DQTrx829212,DQProduk-019,30092025,15:00:55,<NA>
5,DQTrx829213,DQProduk-019,30092025,16:06:16,<NA>
6,DQTrx829235,DQProduk-019,30092025,17:49:03,<NA>
7,DQTrx829167,DQProduk-024,30092025,21:51:24,<NA>
8,DQTrx829168,DQProduk-025,30092025,02:05:10,<NA>
9,DQTrx829254,DQProduk-025,30092025,08:12:00,<NA>


In [74]:
#imputasi
data_transaksi['units'] = data_transaksi['units'].fillna(0)
data_transaksi[data_transaksi['units'].isna()]

,trx_id,product_id,trx_date,trx_time,units


In [75]:
#kolom yang kosong diisi dengan 0
data_transaksi[data_transaksi['units']==0]

,trx_id,product_id,trx_date,trx_time,units
0,DQTrx829214,DQProduk-016,30092025,01:18:05,0
1,DQTrx829166,DQProduk-017,30092025,18:58:33,0
2,DQTrx829118,DQProduk-018,30092025,06:58:09,0
3,DQTrx829186,DQProduk-019,30092025,09:28:13,0
4,DQTrx829212,DQProduk-019,30092025,15:00:55,0
5,DQTrx829213,DQProduk-019,30092025,16:06:16,0
6,DQTrx829235,DQProduk-019,30092025,17:49:03,0
7,DQTrx829167,DQProduk-024,30092025,21:51:24,0
8,DQTrx829168,DQProduk-025,30092025,02:05:10,0
9,DQTrx829254,DQProduk-025,30092025,08:12:00,0


In [91]:
#duplikasi hasil dari dibawah ini tidak ada subset yang di duplikasi
data_transaksi.duplicated(subset = ['trx_id']).sum()

np.int64(0)

In [92]:
#cek apakah ada dua data row yg sama
data_transaksi.duplicated().sum()

np.int64(0)

In [76]:
#cleaning data menghapus mata uang agar cost bisa dialkukan perhitungan secara numerik
data_product

,product_id,product_name,product_category,product_cost,product_price
0,DQProduk-009,Pulpen Metal DQLab,Alat Tulis,IDR 149850,IDR 164850
1,DQProduk-003,Sticky Notes DQLab,Alat Tulis,IDR 29850,IDR 59850
2,DQProduk-025,Tipe-X Roller DQLab,Alat Tulis,IDR 29850,IDR 44850
3,DQProduk-016,Set Pulpen DQLab,Alat Tulis,IDR 44850,IDR 149850
4,DQProduk-028,Penanda Warna DQLab,Alat Tulis,IDR 59850,IDR 164850
5,DQProduk-004,Notebook A5 DQLab,Buku Catatan,IDR 149850,IDR 194850
6,DQProduk-006,Earphone Kantor DQLab,Elektronik,IDR 104850,IDR 224850
7,DQProduk-021,Kalkulator Besar DQLab,Elektronik,IDR 104850,IDR 149850
8,DQProduk-032,Timbangan Mini DQLab,Elektronik,IDR 179850,IDR 224850
9,DQProduk-013,Headset Kantor DQLab,Elektronik,IDR 224850,IDR 314850


In [79]:
data_product.head()

,product_id,product_name,product_category,product_cost,product_price
0,DQProduk-009,Pulpen Metal DQLab,Alat Tulis,IDR 149850,IDR 164850
1,DQProduk-003,Sticky Notes DQLab,Alat Tulis,IDR 29850,IDR 59850
2,DQProduk-025,Tipe-X Roller DQLab,Alat Tulis,IDR 29850,IDR 44850
3,DQProduk-016,Set Pulpen DQLab,Alat Tulis,IDR 44850,IDR 149850
4,DQProduk-028,Penanda Warna DQLab,Alat Tulis,IDR 59850,IDR 164850


In [90]:
#setelah di cleaning
for col in ['product_cost', 'product_price']:
 if data_product[col].dtype == 'object':
  data_product[col] = data_product[col].str.replace('IDR ','')
  data_product[col] = data_product[col].astype(pd.Int64Dtype())

data_product.head()

,product_id,product_name,product_category,product_cost,product_price
0,DQProduk-009,Pulpen Metal DQLab,Alat Tulis,149850,164850
1,DQProduk-003,Sticky Notes DQLab,Alat Tulis,29850,59850
2,DQProduk-025,Tipe-X Roller DQLab,Alat Tulis,29850,44850
3,DQProduk-016,Set Pulpen DQLab,Alat Tulis,44850,149850
4,DQProduk-028,Penanda Warna DQLab,Alat Tulis,59850,164850


In [93]:
data_product.duplicated().sum()
#hasilnya ada 2 data yang sam persis

np.int64(2)

In [96]:
data_product[data_product.duplicated(subset='product_id', keep=False)]
#hasilnya ada beberapa data yang sama/duplikasi

,product_id,product_name,product_category,product_cost,product_price
17,DQProduk-035,Kartu Ucapan DQLab,Kertas & Cetak,59850,119850
18,DQProduk-035,Kartu Ucapan DQLab,Kertas & Cetak,59850,119850
27,DQProduk-015,Mousepad Custom DQLab,Merchandise,59850,89850
28,DQProduk-015,Mousepad Custom DQLab,Merchandise,59850,89850


In [98]:
data_product.shape

(35, 5)

In [97]:
data_product = data_product.drop_duplicates(subset='product_id', keep='first')
data_product.shape
#hasilnya menghapus baris 18 dan 28

(35, 5)

In [99]:
#join menggabungkan 2 tabel transaksi
data_integration=data_transaksi.merge(data_product, on='product_id', how='left')
data_integration.head()

,trx_id,product_id,trx_date,trx_time,units,product_name,product_category,product_cost,product_price
0,DQTrx829214,DQProduk-016,30092025,01:18:05,0,Set Pulpen DQLab,Alat Tulis,44850,149850
1,DQTrx829166,DQProduk-017,30092025,18:58:33,0,Set Tulis Eksekutif DQLab,Merchandise,209850,299850
2,DQTrx829118,DQProduk-018,30092025,06:58:09,0,Printer Mini DQLab,Elektronik,524850,599850
3,DQTrx829186,DQProduk-019,30092025,09:28:13,0,Kertas Warna DQLab,Kertas & Cetak,209850,239850
4,DQTrx829212,DQProduk-019,30092025,15:00:55,0,Kertas Warna DQLab,Kertas & Cetak,209850,239850


## **Proses Simpan Data (Load)**

In [100]:
import pandas as pd

# Upload ke BigQuery - tabel belum ada
to_gbq(
    data_integration,
    destination_table = 'tbl_dwh_dqcommerce.tbl_integration',
    project_id = project_id,
    if_exists = 'fail'
)

100%|██████████| 1/1 [00:00<00:00, 8322.03it/s]


---

### Overview Total Penjualan Periode Data Tiap Bulan

In [ ]:
data_transaksi['month_year'] = data_transaksi['trx_date'].dt.strftime('%Y-%m')
data_transaksi.head()

,trx_id,product_id,trx_date,trx_time,units,month_year
0,DQTrx829214,DQProduk-016,2025-09-30,01:18:05,<NA>,2025-09
1,DQTrx829166,DQProduk-017,2025-09-30,18:58:33,<NA>,2025-09
2,DQTrx829118,DQProduk-018,2025-09-30,06:58:09,<NA>,2025-09
3,DQTrx829186,DQProduk-019,2025-09-30,09:28:13,<NA>,2025-09
4,DQTrx829212,DQProduk-019,2025-09-30,15:00:55,<NA>,2025-09


In [ ]:
penjualan_bulanan = data_transaksi.groupby(['month_year'], as_index = False).agg(total = ('units', 'sum'))
penjualan_bulanan.head()

,month_year,total
0,2024-01,38009
1,2024-02,36935
2,2024-03,39981
3,2024-04,47102
4,2024-05,46910


In [ ]:
import plotly.express as px

# Membuat line chart
fig = px.line(
    penjualan_bulanan,
    x="month_year",
    y="total",
    title="<b>Grafik Total Penjualan Bulanan DQ-Commerce</b>",
    color_discrete_sequence = ['#39074E'],
    markers=False)

"""
fig.add_vrect(
    x0="2025-03", x1="2025-09",
    fillcolor="#E73c35",
    opacity=0.5,
    layer="below",
    line_width=0,
    #annotation_text="Tahun 2025",
    #annotation_position="top "
)
"""

fig.update_layout(
    height = 450,
    width = 1100,
    plot_bgcolor = 'rgba(0, 0, 0, 0)',
    yaxis = dict(
        showline = True,
        linewidth = 1,
        color = '#544018',
        linecolor = '#544018',
        showgrid=False,
    ),
    xaxis = dict(
        title = '',
        showline = True,
        linewidth = 1,
        color = '#544018',
        linecolor = '#544018',
        showgrid=False,
    ),
)

fig.show()

In [ ]:
penjualan_per_produk = data_transaksi.groupby(['month_year', 'product_id'], as_index = False).agg(total = ('units', 'sum'))
penjualan_per_produk.head()

,month_year,product_id,total
0,2024-01,DQProduk-001,3441
1,2024-01,DQProduk-002,1513
2,2024-01,DQProduk-003,76
3,2024-01,DQProduk-004,166
4,2024-01,DQProduk-005,276


In [ ]:
import plotly.express as px

# Membuat line chart
fig = px.line(
    penjualan_per_produk, #[penjualan['month_year'] >= '2025-01'],
    x="month_year",
    y="total",
    color="product_id",
    title="<b>Grafik Total Penjualan Bulanan DQ-Commerce Per Produk</b>",
    markers=True
)

fig.update_layout(
    height = 550,
    width = 1200,
    plot_bgcolor = 'rgba(0, 0, 0, 0)',
    yaxis = dict(
        showline = True,
        linewidth = 1,
        color = '#544018',
        linecolor = '#544018',
        showgrid=False,
    ),
    xaxis = dict(
        title = '',
        showline = True,
        linewidth = 1,
        color = '#544018',
        linecolor = '#544018',
        showgrid=False,
    ),
)

fig.show()

In [ ]:
import plotly.graph_objects as go

highlight_product = ['DQProduk-003', 'DQProduk-019', 'DQProduk-025', 'DQProduk-018', 'DQProduk-031']

fig = go.Figure()

# Loop setiap produk
for product in penjualan_per_produk['product_id'].unique():
    df_product = penjualan_per_produk[penjualan_per_produk['product_id'] == product]

    # Tentukan properti styling
    if product in highlight_product:
        line_color = 'purple'
        line_width = 3
    else:
        line_color = 'lightgray'
        line_width = 1.2

    # Tambahkan trace
    fig.add_trace(go.Scatter(
        x=df_product['month_year'],
        y=df_product['total'],
        mode='lines+markers',
        name=product,
        line=dict(
            width=line_width,
            color=line_color
        )
    ))

# Layout styling
fig.update_layout(
    title="<b>Grafik Total Penjualan Bulanan DQ-Commerce Per Produk</b>",
    height=550,
    width=1200,
    plot_bgcolor='rgba(0, 0, 0, 0)',
    xaxis=dict(
        showline=True,
        linewidth=1,
        color='#544018',
        linecolor='#544018',
        title='',
        showgrid=False,
    ),
    yaxis=dict(
        showline=True,
        linewidth=1,
        color='#544018',
        linecolor='#544018',
        showgrid=False,
    )
)

fig.show()